In [1]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('hf_token')"

In [2]:
!git clone https://github.com/cutefluffyfox/MT-vocab-tuning.git

Cloning into 'MT-vocab-tuning'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 245 (delta 141), reused 182 (delta 82), pack-reused 0 (from 0)
Receiving objects: 100% (245/245), 523.00 KiB | 14.94 MiB/s, done.
Resolving deltas: 100% (141/141), done.


In [3]:
!pip install yadisk[sync_defaults]
!pip install transformers==4.49.0 sacremoses~=0.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 86.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 41.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [4]:
import yadisk
client = yadisk.Client(token="ya_token")

In [5]:
import os
import sys

# change to upper dir to use all custom libs (won't be needed if run from main scripts)
sys.path.insert(1, os.path.join(os.getcwd(), 'MT-vocab-tuning'))

In [6]:
# MAIN TRAINIG PARAMETERS ALL IN ONE PLACE
nllb_model: str            = 'facebook/nllb-200-distilled-1.3B'
lang: str                  = 'mhr'
update_tokenizer: bool     = True
add_token_lang: bool       = True
learn_both_direction: bool = False
ttl: int                   = 4 * 60 * 60  # training ttl (in seconds)

# if not present in dict -> will copy value from the same (default script type is Cyrl)
lang_to_simmilar_lang_map = {
    'mhr': 'est_Latn',  # both finno-ugric
    'tat': 'tat_Cyrl',
}

In [7]:
from scripts.helpers.model_manager import NLLB200Model

model = NLLB200Model(device='cuda')

model.translate_batch(['Счастливые лисички прыгают прыг скок', 'раз два три!'], 'ru', 'kaz')

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

['Счастливые лисички прыгают прыгают скак Бақытты лисички секіріп секіріп секіреді',
 'Бір рет екі рет үш рет.']

In [8]:
import pandas as pd
from scripts.helpers import dataset_manager


if lang == 'tat':
    ds = dataset_manager.HFDataset('tat-ru', 'ipsan.tatar-parallel-400k')
    ds.download_json("hf://datasets/IPSAN/tatar_translation_dataset/train.json")
    tt_ru = ds.get_df()
    
    tt_ru_parsed = []
    for row in tt_ru.to_dict('records'):
        tt_ru_parsed.append(eval(row['translation']))
    df = pd.DataFrame(tt_ru_parsed)
    df = df.rename(columns={'ru': 'rus', 'tt': 'tat'})
    df
elif lang == 'mhr':
    ds = dataset_manager.HFDataset('mhr-ru', 'mari-parallel-413k')
    ds.download_parquet("hf://datasets/AigizK/mari-russian-parallel-corpora/data/train-00000-of-00001.parquet")
    mhr_ru = ds.get_df()
    
    df = mhr_ru
    df = df.rename(columns={'ru': 'rus', 'chm': 'mhr'})
    df

In [9]:
from scripts.helpers.tokenizer_manager import add_new_full_language, initialize_new_model_emb

tokenizer = add_new_full_language(
    model_name=nllb_model, 
    data=df if update_tokenizer else None,
    lang=lang, 
    add_token_lang=add_token_lang
)


100%|██████████| 400000/400000 [00:37<00:00, 10709.81it/s]


Nllb tokens set len: 256000
<pad> not in nllb tokens set?
This is UNEXPECTED BEHAVIOR that cause to BREAK tokenizer
We have hotfix to just skip, but IT IS BAD if you are seeing this
Old tokenizer length: 256204
New tokenizer length: 268478
Code of `tat_Cyrl`: 268447
Code of `<mask>`: 268275
Decoded `tat_Cyrl` and `<mask>`: ['tat_Cyrl', '<mask>']


In [10]:
print('Sanity check that tokenizer is continious')
indexes = set(list(range(len(tokenizer))))
for k, v in tokenizer.get_vocab().items():
    if v not in indexes:
        print('not:', k, v)
    else:
        indexes -= {v}
print('Skipped indexes:', indexes)
print('All good? ', 'no!' if indexes else 'yes')
if indexes:
    raise ValueError('YOU TOKENIZER IS NOT CONTINIOUS, PLEASE FIX IT')

Sanity check that tokenizer is continious
Skipped indexes: set()
All good?  yes


In [12]:
# swap tokenizer to new one
model.tokenizer = tokenizer
initialize_new_model_emb(
    model_name=nllb_model, 
    model=model.model, 
    tokenizer=model.tokenizer, 
    lang=lang, 
    similar_lang=lang_to_simmilar_lang_map.get(lang, lang),
)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Amount of moved tokens: 203 ['tso_Latn', 'lij_Latn', 'tzm_Tfng', 'ars_Arab', 'pes_Arab', 'hrv_Latn', 'lmo_Latn', 'pbt_Arab', 'kbp_Latn', 'zul_Latn']
Previous tokenizer length: 256204
New tokenizer length: 268478
Ids of new tokens: min [256001], max [268274], mean [262137]
Amount of new tokens: 12274
['▁лоббистлар', '▁ителеше', '▁кайдадыр', '▁Новости', 'Мәдәният', '▁күзәт', '▁уйламый', '▁труппа', 'экс', '▁фаразлаган', '▁кайтачак', 'Ил', 'Коммерция', '▁бәхәс', '▁вазифасыннан']


100%|██████████| 12274/12274 [00:03<00:00, 3477.34it/s]


In [13]:
# check again everything is normal
model.translate_batch(['Счастливые лисички прыгают прыг скок', 'раз два три!'], 'ru', lang)

['', 'a a a a a a']

In [14]:
model.train(
    experiment_name=f'ru-{lang}.{"both_directions" if learn_both_direction else "single_direction"}.{ds.name}.nllb.{"update_tokenizer" if update_tokenizer else "no_upd_tokenizer"}',
    data=df,
    src_lang='rus',
    dst_lang=lang,
    learn_both_direction=learn_both_direction,
    training_steps=100_000,      # we are more likely to get ttl'ed
    save_every=600,              # on local [2080 super mobile] run it took ~24minutes for 100 it's. So each 300 is around 1 ckpt/h.
    ttl=ttl,                     # 4h
    ya_client=client,
)

  0%|          | 0/100000 [00:00<?, ?it/s]

0 4.96875


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


KeyboardInterrupt: 